# CSC299: Summer 2021: Report 1 - Implementation of LCU with 1 Ancilla

Arkaprava Choudhury

This shall be the first report of the CSC299 project about the automatized implementations of nonunitary operations using quantum algorithms. This report is meant to serve as an introduction to the concept of implementing any operation using its decomposition as a linear combinations of unitaries.

In [1]:
import tequila as tq
import copy
from typing import Iterable
import numpy as np

# Introduction

Quantum computing refers to using the properties of quantum states to perform computation; popular examples of such properties include superposition, interference and entanglement. It is believed that quantum computing can enable more efficient algorithms for certain problems than can be possible using solely classical methods, and one such supposed application involves the simulation of quantum systems.

Over the past thirty years, multiple developments have been made in this field which demonstrate that quantum computers can efficiently simulate Hamiltonian dynamics and much of current ongoing work involves improving the performance of these methods and expanding the scope of such simulations.

We wish to have a method that can simulate the effect of running any non-unitary operation $H$, if we know how to write it as a sum of unitary operations $U_j$ if we already have some mechanisms for implementing each of the $U_j$.

One of the many practical uses of such a technique is as follows. Suppose that we wish to simulate the evolution under some Hamiltonian $H$ for time $t$ within a maximum error of some $\epsilon > 0$ with the operator $U = e^{-iHt}$. Divide the time $t$ into $r$ equal time-segments of length $t/r$. For each segment, define the operator for time-evolution within that segment as follows:
\begin{align*}
    U_r = e^{-iHt/r} \approx \sum_{j=0}^K \frac1{j!} (-iHt/r)^j
\end{align*}

Here, we wish to truncate the Taylor series expansion of the exponential at some order $K$ such that the maximum error for each segment is $\epsilon/r$, which would imply that the maximum result across all the segments is $\epsilon$. Assuming that $r > \left|\left| H \right|\right| t$ (where $\left|\left| \cdot \right|\right|$ denotes the usual matrix norm), we see that we must have:
\begin{align*}
    K &\in \mathcal{O}\left( \frac{\log(r/\epsilon)}{\log(\log(r/\epsilon))} \right)
\end{align*}

# Procedure

Consider the Hamiltonian $H$ which acts over $n$ qubits. Let $N = 2^n$. Without loss of generality, we can assume that $H$ is Hermitian. Indeed, suppose that $H$ was not Hermitian. Then, we would define $\tilde{H}$ such that:
\begin{align*}
    \tilde{H} &=
    \begin{bmatrix}
        0 & H^\dagger \\
        H & 0
    \end{bmatrix}
\end{align*}
Thus, $\tilde{H}$ acts over $2n$ qubits, where the first $n$ qubits are ancillae. Furthermore, as is evident by the construction above, $\tilde{H}$ is Hermitian, even though $H$ is not.

Suppose that we have the decomposition $H = \sum_{j=0}^{m-1} \alpha_j U_j$ over $n$ qubits, where each of the $U_j$'s are unitary. We know that any arbitrary Hamiltonian can be expressed in such a form due to the fact that any element of the standard orthonormal basis can be expressed as a linear combination of such unitary operators (see Appendix for detailed proof).

Let $s = \sum_{k=0}^{m-1} \alpha_k$. Moreover, without loss of generality, we can assume that $\alpha_j > 0$ for all $j$. Indeed, suppose that there exists some $\alpha_j < 0$: we can then replace the term $\alpha_j U_j$ with the term $\alpha^\prime_j U^\prime_j$, where $(\alpha^\prime_j, U^\prime_j) = (-\alpha_j, - U_j)$.

Given $m$ unitaries in the expansion of $H$, we shall require $\lceil\log_2 {m}\rceil$ qubits in the ancillary register, along with as many qubits as required by $H$ for the state register. We define two oracles: the Prepare oracle shall only act on the ancilla, while the Select operator shall act on both the ancillary and state registers.

We define the $\text{Prepare}$ oracle such that:
\begin{align*}
    \text{Prepare}\left| 0 \right\rangle &= \frac1{\sqrt{s}} \sum_{j=0}^{m-1} \sqrt{\alpha_j} \left| j \right\rangle
\end{align*}
We also define the $\text{Select}$ oracle as follows:
\begin{align*}
    \text{Select}\left| j \right\rangle \left| \psi \right\rangle &= \left| j \right\rangle U_j \left| \psi \right\rangle
\end{align*}

We thus define the operator $W$ as follows:
\begin{align*}
    W &= (\text{Prepare}^\dagger\otimes I_N) \text{Select} (\text{Prepare}\otimes I_N)
\end{align*}
Therefore, we see that:
\begin{align*}
    W \left| 0 \right\rangle \left| \psi \right\rangle &= (\text{Prepare}^\dagger\otimes I_N) \text{Select} (\text{Prepare}\otimes I_N) \left| 0 \right\rangle \left| \psi \right\rangle \\
    &= (\text{Prepare}^\dagger\otimes I_N) \text{Select} \left( \frac1{\sqrt{s}} \sum_{j=0}^{m-1} \sqrt{\alpha_j} \left| j \right\rangle \left| \psi \right\rangle \right) \\
    &= (\text{Prepare}^\dagger\otimes I_N) \left( \frac1{\sqrt{s}} \sum_{j=0}^{m-1} \sqrt{\alpha_j} \left| j \right\rangle U_j \left| \psi \right\rangle \right) \\
    &= \frac1{s} \left| 0 \right\rangle H\left| \psi \right\rangle + \sqrt{1-\frac1{s^2}} \left| \phi \right\rangle
\end{align*}
where $\left| \phi \right\rangle$ is some state whose ancillary component lies in a subspace orthogonal to $\left| 0 \right\rangle$. Hence, if $P_0 = \left| 0 \right\rangle \left\langle 0 \right|\otimes I_N$ is the projector onto the zero-subspace of the ancillary register, then we have that:
\begin{align*}
    P_0W\left| 0\right\rangle\left| \psi\right\rangle &= \frac1{s} \left| 0 \right\rangle H\left| \psi \right\rangle
\end{align*}
This implies that if we measure the ancillary register after applying the $W$ operator to $\left| 0 \right\rangle \left| \psi \right\rangle$ and we find that the ancillary qubits are all in the 0 state, then we trigger the succesful preparation in the state register, as desired.

The quantum circuit for this procedure looks as shown below:

TODO: ADD DIAGRAM

To obtain an exact implementation of $H$, we can choose to use amplitude amplification at the end of this procedure.

For more details about these equations, refer Appendix.

### Notes on procedure

Notice that when $H$ can be written as a linear combination of $m$ unitary operators with all positive coefficients, then we need a total of $k = \lceil \log_2 m\rceil$ ancillary qubits to implement the Prepare operator described in the above procedure. This is because we need enough qubits in the ancillary register to define a unique labeling for each unitary in the linear combination.

Also note that the success probability of emulating $H$ by the above procedure is $\frac1{s^2}$. This probability depends on the size of the ancilla and decreases exponentially in the number of qubits contained in the ancilla. We can apply the procedure of amplitude amplification (refer Appendix) to increase this probability up to 1. Ideally, if $s=2$, then this fits the procedure of oblivious amplitude amplification. However, since $H$ is not unitary itself, we need to slightly alter the approach of oblivious amplitude amplification that we use here.

Firstly, define the $k$-dimensional ancillary reflection $R = I_N - 2 P_0$ about the zero-state of the ancillary register. Note that this operation is called a reflection because of the way that it operates with respect to elements in the zero state in the ancillary register. For instance, consider the $\left|j\right\rangle$ for the ancillary register where $j\neq0$ and define the state $\left|\phi\right\rangle = \left|j\right\rangle \left|\psi\right\rangle$. Then, we have that:
\begin{align*}
    R \left|\phi\right\rangle &= (I_k - 2 P_0) \left|\phi\right\rangle \\
    &= I_k \left|\phi\right\rangle - 2 P_0 \left|\phi\right\rangle \\
    &= \left|\phi\right\rangle - 0 \\
    &= \left|\phi\right\rangle
\end{align*}
However, now consider the state $\left|0\right\rangle$ for the ancillary register, and consider $\left|\phi\right\rangle = \left|0\right\rangle \left|\psi\right\rangle$. Then, we have that:
\begin{align*}
    R \left|\phi\right\rangle &= (I_k - 2 P_0) \left|\phi\right\rangle \\
    &= I_k \left|\phi\right\rangle - 2 P_0 \left|\phi\right\rangle \\
    &= \left|\phi\right\rangle - 2 \left|\phi\right\rangle \\
    &= - \left|\phi\right\rangle
\end{align*}

Next, define the amplitude amplification operator $A = - WRW^\dagger RW$.

Recall the following facts. Firstly, note that $P_0^2 = P_0$ since $P_0$ is a projection operator and that $P_0 \left| 0 \right\rangle \left| \psi \right\rangle = \left| 0 \right\rangle\left| \psi \right\rangle$. Also note that, by definition, $W$ is a unitary operator. Moreover, recall our assumption that $H$ is Hermitian. Finally, by construction, we have that:
\begin{align*}
    P_0 WP_0 &= \frac1{s} (\left| 0 \right\rangle\left\langle 0 \right|\otimes H)
\end{align*}

Therefore, we observe that:
\begin{align*}
    P_0 A\left| 0 \right\rangle\left| \psi \right\rangle &= \\
    &= \left(\frac3{s}-\frac4{s^3}\right) \left| 0 \right\rangle H \left| \psi \right\rangle
\end{align*}

### One qubit example

Suppse that we are given a Hamiltonian $H$ as follows:
\begin{align*}
    H &=
    \begin{bmatrix}
        0 & 0 \\
        0 & 1
    \end{bmatrix}
\end{align*}
Then, clearly, $H$ is non-unitary, since $HH^\dagger\neq I_2$. However, we can write $H = \frac1{2} I - \frac1{2} Z$, thus expressing $H$ as the sum of two unitary operations.

TODO: Finish working through this example.

# Implementation of one qubit ancilla 

In this section, we wish to create an implementation of the LCU procedure using tequila.

## Implementing Select operator

We first need to define a subroutine to return the controlled version of the unitary. We do so by naturally translating the definition of the Select operator into Python, as follows. We first need to create a way of adding extra controls into each unitary operation, as required by the algorithm, since this is not a part of Tequila's built-in features at the moment. To do so, we construct the following function which returns a new circuit with additional control qubits.

In [2]:
def control_unitary(ancilla, unitary: tq.QCircuit) -> tq.QCircuit:
    """Return controlled version of unitary

    SHOULD NOT mutate unitary

    Preconditions:
        - ancilla and unitary have no common qubits
    """
    gates = unitary.gates
    cgates = []
    for gate in gates:
        cgate = copy.deepcopy(gate)
        if isinstance(ancilla, Iterable):
            control_lst = list(cgate.control) + list(ancilla)
        else:
            control_lst = list(cgate.control) + [ancilla]
        cgate._control = tuple(control_lst)
        cgate.finalize()
        cgates.append(cgate)

    return tq.QCircuit(gates=cgates)

The select oracle is naturally translated into code as follows.

In [4]:
def select_1ancilla(ancillary, unitary_0: tq.QCircuit, unitary_1: tq.QCircuit) -> tq.QCircuit:
    """
    Select operator, when the Hamiltonian can be expressed as the linear combination of two
    unitary operators.
    Requires only one ancillary qubit.

    Returns the select oracle.
    """
    impl_1 = control_unitary(ancilla=ancillary, unitary=unitary_1)
    
    x_gate = tq.gates.X(target=ancillary)
    control = control_unitary(ancilla=ancillary, unitary=unitary_0)

    impl_0 = x_gate + control + x_gate

    return impl_1 + impl_0

## Implementing Prepare operator

Next, we implement the prepare oracle for the single-qubit ancilla case in the form of an $R_y$ rotation as follows.

TODO: Add mathematical explanation for prepare_2unitary

We are given the values of $\alpha_0, \alpha_1$ in the expansion, and we wish to create a circuit that can be expressed as follows:
\begin{align*}
    \mathrm{Prep} &=
    \begin{bmatrix}
        \sqrt{\frac{\alpha_0}{\alpha_0 + \alpha_1}} & \sqrt{\frac{\alpha_1}{\alpha_0 + \alpha_1}} \\
        x_0 & x_1
    \end{bmatrix}
\end{align*}
Here, $x_0$ and $x_1$ denote unknown elements of the matrix with which we are not concerned about. This is because the $\mathrm{Prep}$ circuit shall only ever be applied to the zero-state on the ancilla, and so, the mapping for the one-state of the ancilla is of no importance.

Now, recall that the general form for a $R_y$ rotation of angle $\theta$ is as follows:
\begin{align*}
    R_y(\theta) &=
    \begin{bmatrix}
        \cos(\theta/2) & -\sin(\theta/2) \\
        \sin(\theta/2) & \cos(\theta/2)
    \end{bmatrix}
\end{align*}
Hence, if we wish to express the $\mathrm{Prep}$ operator in the form of a $R_y$ rotation, we would hvae the following:
\begin{align*}
    \cos(\theta/2) &= \sqrt{\frac{\alpha_0}{\alpha_0 + \alpha_1}} \\
    -\sin(\theta/2) &= \sqrt{\frac{\alpha_1}{\alpha_0 + \alpha_1}}
\end{align*}
This implies that:
\begin{align*}
    \theta &= -2 \cdot \arcsin\left( \sqrt{\frac{\alpha_1}{\alpha_0 + \alpha_1}} \right)
\end{align*}

Therefore, we now see that $\mathrm{Prep} = R_y \left( -2 \cdot \arcsin\left( \sqrt{\frac{\alpha_1}{\alpha_0 + \alpha_1}} \right) \right)$ and we can naturally translate this into Python as shown in the following cell.

In [8]:
def prepare_1ancilla(ancillary, sum_of_unitaries: list[tuple[float, tq.QCircuit]]) -> tq.QCircuit:
    """
    Prepare operator, when the Hamiltonian can be expressed as the linear combination of two
    unitary operators.
    Requires only one ancillary qubit.

    Preconditions:
        - alpha_0 > 0 and alpha_1 > 0
    """
    alpha_0, alpha_1 = sum_of_unitaries[0][0], sum_of_unitaries[1][0]

    theta = -2 * np.arcsin(np.sqrt(alpha_1 / (alpha_0 + alpha_1)))

    return tq.gates.Ry(target=ancillary, angle=theta)

In [5]:
def lcu_1ancilla(unitaries: list[tuple[float, tq.QCircuit]]) -> tq.QCircuit:
    """Return the circuit (Prep)(Select)(Prep.dagger()) according to the LCU algorithm,
    with 0 as the ancilla qubit.
    
    Preconditions:
        - all(0 not in unitary.qubits() for unitary in [u[1] for u in unitaries])
    """
    prepare = prepare_2unitary(0, unitaries)
    circ = prepare + select_2unitary(0, unitaries[0][1], unitaries[1][1]) + prepare.dagger()
    return circ

### Notes on implementation

Input, output as circuits. Advantage: independent of starting state, so if initial state can be efficiently prepared, then we can efficiently simulate Hamiltonian. Drawback: need to have the circuits for each unitary used prepared beforehand.

### Example

In [11]:
def example_function() -> tq.QCircuit:
    """Test example for LCU with two unitary matrices"""
    identity = tq.QCircuit()
    unitaries = [(0.5, identity), (0.5, tq.gates.Z(1))]
    return algorithm_2unitary(unitaries=unitaries)

In [12]:
result = example_function()
tq.draw(result, backend='qiskit')

┌──────────┐   ┌───┐┌───┐┌─────────┐
q_0: ┤ RY(-π/2) ├─■─┤ X ├┤ X ├┤ RY(π/2) ├
     └──────────┘ │ └───┘└───┘└─────────┘
q_1: ─────────────■──────────────────────
                                         
c: 2/════════════════════════════════════

# Amplitude amplification

TODO

# Testing

As always, it is a good principle to always test out the functions and ensure that they work for all cases, including any possible edge cases that might exist. To do so, often the best way is to create randomized testing functions and repeatedly run the algorithm on such randomized examples to be confident that the algorithm works for any given case.

TODO: Property-based testing

# Next steps: 3 or 4 unitaries (2 ancillae), and then, m unitaries